# Visualizing mutual fund portfolio overlaps with other indices and mutual fund portfolios

We had discussed Upset charts for visualizing fund portfolio overlaps in [Mutual fund portfolio overlap in UpSet charts](https://stockviz.biz/2018/10/22/mutual-fund-portfolio-overlap-in-upset-charts/). You can use the code below to roll your own overlap analysis for the funds you care about.

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(UpSetR)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()
mfi <- MutualFundsIndia()

The *Franklin India Prima* fund is a mid-cap fund. We will see how the portfolio is actually setup by intersecting it with both large-cap and mid-cap index portfolios.

In [ ]:
indexName1 <- "NIFTY 100" #large-cap universe
indexName2 <- "NIFTY MIDCAP 150" #mid-cap universe
schemeCode <- 100473
schemeName <- "Franklin India Prima"
portfolioDate <- as.Date("2019-05-31")

In [ ]:
# grab the data
mfPortfolio <- mfi$Portfolio() %>%
    filter(SCHEME_CODE == schemeCode 
           & INSTRUMENT == 'EQUITY' 
           & !is.null(WEIGHTAGE)
           & WEIGHTAGE > 0
           & PORTFOLIO_DATE == portfolioDate
           & (INSTRUMENT_TYPE == 'STOCK' 
              | INSTRUMENT_TYPE == 'E' 
              | INSTRUMENT_TYPE == '' 
              | is.null(INSTRUMENT_TYPE))) %>%
    select(SYMBOL, WEIGHTAGE) %>%
    arrange(desc(WEIGHTAGE)) %>%
    collect()

indexPort1 <- indices$NseConstituents() %>%
    filter(NAME == indexName1) %>%
    select(SYMBOL, CAP_WEIGHT) %>%
    arrange(desc(CAP_WEIGHT)) %>%
    collect()

indexPort2 <- indices$NseConstituents() %>%
    filter(NAME == indexName2) %>%
    select(SYMBOL, CAP_WEIGHT) %>%
    arrange(desc(CAP_WEIGHT)) %>%
    collect()

mfPortfolio %>% print(n=5)
indexPort1 %>% print(n=5)
indexPort2 %>% print(n=5)

In [ ]:
# create the chart

#UpSetR takes a list
listInput <- list(mfPortfolio$SYMBOL, indexPort1$SYMBOL, indexPort2$SYMBOL)
names(listInput) <- c(schemeName, indexName1, indexName2)

upset(fromList(listInput), order.by='degree', nsets=length(listInput), keep.order=T, mainbar.y.label='Portfolio Intersections', sets.x.label ='Portfolio Size', text.scale=2)
grid.text(sprintf("Overlap: %s/%s/%s", schemeName, indexName1, indexName2),x = 0.1, y=0.95, just='left', gp=gpar(fontsize=20, fontfamily='Segoe UI'))
grid.text("@StockViz",x = 0.1, y=0.8, rot=90, gp=gpar(fontsize=14, fontfamily='Segoe UI', col='grey'))

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)